In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

cur_dir = os.getcwd()
cur_dir

'/scratch/garrettroell/machine_learning_clostridium/slope_prediction_lib'

In [2]:
raw_data = pd.read_csv(f'{cur_dir}/processed_data/raw_data.csv')
raw_data.set_index(['composition','trial','time'],drop=True,inplace=True)

smooth_data = pd.read_csv(f'{cur_dir}/processed_data/smooth_data.csv')
smooth_data.set_index(['composition','trial','time'],drop=True,inplace=True)

In [3]:
display(raw_data.head())
display(smooth_data.head())

acetate  biomass  butanol  butyrate  ethanol  \
composition trial time                                                 
1           1     0.58    21.61     0.41     0.04      0.06    10.94   
                  0.65    44.31     0.39     0.05      0.08    15.89   
                  1.02    46.19     0.46     0.21      0.64     8.14   
                  1.67    46.16     0.49     1.18      3.64    10.81   
                  3.70    34.39     0.64     8.44      9.76    20.34   

                        flow rate     H2   CO    CO2   acetate_Δ  biomass_Δ  \
composition trial time                                                        
1           1     0.58         20  0.125  0.5  0.375    0.000000   0.000000   
                  0.65         20  0.125  0.5  0.375  324.285714  -0.285714   
                  1.02         20  0.125  0.5  0.375    5.081081   0.189189   
                  1.67         20  0.125  0.5  0.375   -0.046154   0.046154   
                  3.70         20  0.125  0.5  0.375   -5.798030   0.073892   

                        butanol_Δ  butyrate_Δ  ethanol_Δ  
composition trial time                                    
1           1     0.58   0.000000    0.000000   0.000000  
                  0.65   0.142857    0.285714  70.714286  
                  1.02   0.432432    1.513514 -20.945946  
                  1.67   1.492308    4.615385   4.107692  
                  3.70   3.576355    3.014778   4.694581

CO    CO2     H2    acetate   biomass   butanol  \
composition trial time                                                     
1           1     0.2   0.5  0.375  0.125  19.831073  0.410803  0.028316   
                  0.3   0.5  0.375  0.125  22.196249  0.403602  0.031357   
                  0.4   0.5  0.375  0.125  25.293925  0.403709  0.028700   
                  0.5   0.5  0.375  0.125  29.005157  0.406214  0.031038   
                  0.6   0.5  0.375  0.125  32.899333  0.410848  0.043533   

                        butyrate    ethanol  flow rate  acetate_Δ  ...  \
composition trial time                                             ...   
1           1     0.2   0.051013  15.144135       20.0  18.298778  ...   
                  0.3   0.050367  14.522308       20.0  23.651756  ...   
                  0.4   0.035514  13.956015       20.0  30.976760  ...   
                  0.5   0.041123  13.322048       20.0  37.112317  ...   
                  0.6   0.081221  12.648214       20.0  38.941765  ...   

                        prev_acetate  prev_biomass  prev_butanol  \
composition trial time                                             
1           1     0.2      18.001195      0.421076      0.028260   
                  0.3      19.831073      0.410803      0.028316   
                  0.4      22.196249      0.403602      0.031357   
                  0.5      25.293925      0.403709      0.028700   
                  0.6      29.005157      0.406214      0.031038   

                        prev_butyrate  prev_ethanol  prev_acetate_Δ  \
composition trial time                                                
1           1     0.2        0.061033      0.028260       10.961665   
                  0.3        0.051013      0.028316       18.298778   
                  0.4        0.050367      0.031357       23.651756   
                  0.5        0.035514      0.028700       30.976760   
                  0.6        0.041123      0.031038       37.112317   

                        prev_biomass_Δ  prev_butanol_Δ  prev_butyrate_Δ  \
composition trial time                                                    
1           1     0.2        -0.084697       -0.015584        -0.144950   
                  0.3        -0.102733        0.000561        -0.100208   
                  0.4        -0.072009        0.030416        -0.006462   
                  0.5         0.001068       -0.026579        -0.148525   
                  0.6         0.025050        0.023385         0.056090   

                        prev_ethanol_Δ  
composition trial time                  
1           1     0.2        -0.144950  
                  0.3        -0.100208  
                  0.4        -0.006462  
                  0.5        -0.148525  
                  0.6         0.056090  

[5 rows x 24 columns]

In [6]:
def get_X_y_arrays_conc(input_data, specific_comps):


    input_data_copy = input_data.copy()
    input_data_copy = input_data_copy.loc[specific_comps]
    input_data_copy.reset_index(inplace=True)
    print(input_data_copy)
    X  = input_data_copy [['time','acetate_0', 'biomass_0', 'butanol_0', 'butyrate_0', 'ethanol_0', 'CO', 'CO2', 'H2', 'flow rate']]
    y = input_data_copy  [['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol']]

    return np.array(X), np.array(y)

In [7]:
from machine_learning.model_selector import model_selector

trained_models = {}

for training_data in ['raw', 'smooth']:
    for regressor in ['gradient boosting', 'random forest', 'support vector', 'neural net', 'lasso']:
        for test_comp in ['none', 1, 2, 3, 4, 5, 6, 7]:
            model_name = regressor + ', ' + training_data + ', test comp = ' + str(test_comp)
            print(model_name)

            # set up training set
            if training_data == 'raw':
                data = raw_data
            else:
                data = smooth_data

            # set up training comps
            training_comps = [1, 2, 3, 4, 5, 6, 7]
            if test_comp != 'none':
                training_comps.remove(test_comp)

            # get input and output arrays
            X, y = get_X_y_arrays_conc(data, training_comps)

            # get ML model to use, and fit it
            model = model_selector(regressor)
            trained_models[model_name] = model.fit(X, y)

gradient boosting, raw, test comp = none
    composition  trial  time  acetate  biomass  butanol  butyrate  ethanol  \
0             1      1  0.58    21.61     0.41     0.04      0.06    10.94   
1             1      1  0.65    44.31     0.39     0.05      0.08    15.89   
2             1      1  1.02    46.19     0.46     0.21      0.64     8.14   
3             1      1  1.67    46.16     0.49     1.18      3.64    10.81   
4             1      1  3.70    34.39     0.64     8.44      9.76    20.34   
..          ...    ...   ...      ...      ...      ...       ...      ...   
69            6      1  4.54    54.81     0.31     4.56     12.77    13.30   
70            7      1  0.65    58.14     0.39     0.00     11.75     3.48   
71            7      1  1.75    57.29     0.50     2.02     15.06     6.30   
72            7      1  3.58    56.61     0.45     9.31     15.67    24.63   
73            7      1  4.79    50.51     0.52    10.72     13.10    38.50   

    flow rate     H2  

KeyError: "['butyrate_0', 'biomass_0', 'butanol_0', 'ethanol_0', 'acetate_0'] not in index"

In [8]:
model = trained_models['gradient boosting, raw, test comp = none']
# model

In [9]:
def get_model_predictions_conc(model_dict, input_df):
    model_predictions = {}
    ml_input, _ = get_X_y_arrays_conc(input_df, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

    for model_name in model_dict.keys():
        model = model_dict[model_name]
        prediction = model.predict(ml_input)
        prediction_df = pd.DataFrame(data=prediction, index=input_df.index, columns=['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol'])
        model_predictions[model_name] = prediction_df

    return model_predictions

In [10]:
measured_time_predictions = get_model_predictions_conc(trained_models, raw_data)
smoothed_time_predictions = get_model_predictions_conc(trained_models, smooth_data)

Define r2, rmse, and normalized rmse methods for evaluating model predictions

In [11]:
from scipy.stats import linregress
from sklearn import metrics

def get_pearson_r2 (measured_list, predicted_list):
    # slope, intercept, r_value, p_value, std_err
    _, _, r_value, _, _ = linregress(measured_list, predicted_list)
    r2 = (r_value**2)
    return r2

def get_rmse (measured_list, predicted_list):
    mse = metrics.mean_squared_error(measured_list, predicted_list)
    rmse = (mse**0.5)
    return rmse

def get_norm_rmse (measured_list, predicted_list):
    mse = metrics.mean_squared_error(measured_list, predicted_list)
    rmse = (mse**0.5)
    avg_meas = sum(measured_list) / len(measured_list) 
    return rmse/avg_meas

Define function to evaluate models

In [12]:
def evaluate_models(pred_df_dict, raw_df, metric):
    species_set = ['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol']
    test_comp_set=[1,2,3,4,5,6,7]
    index_set = ['gradient boosting, raw', 'random forest, raw', 'support vector, raw', 'neural net, raw', 'lasso, raw', 'gradient boosting, smooth', 'random forest, smooth', 'support vector, smooth', 'neural net, smooth', 'lasso, smooth']
    
    for species in species_set:
        data = {}
        for test_comp in test_comp_set:
            data[test_comp] = []
            for model_name in pred_df_dict.keys():
                # print(model_name)
                if str(test_comp) in model_name:
                    predicted_species_values = list(pred_df_dict[model_name].loc[test_comp][species])
                    measured_species_values = list(raw_df.loc[test_comp][species])

                    r2 = get_pearson_r2(measured_species_values, predicted_species_values)
                    rmse = get_rmse (measured_species_values, predicted_species_values)
                    norm_rmse = get_norm_rmse (measured_species_values, predicted_species_values)
                    
                    if metric == 'r2':
                        data[test_comp].append(r2)
                    elif metric == 'rmse':
                        data[test_comp].append(rmse)
                    elif metric == 'norm_rmse':
                        data[test_comp].append(norm_rmse)
                    else:
                        print('unknown metric')
        species_data = pd.DataFrame.from_dict(data)
        species_data[f'model for {species}'] = index_set
        species_data.set_index(f'model for {species}', inplace=True, drop=True)
        display(species_data)

In [13]:
evaluate_models(smoothed_time_predictions, smooth_data, 'r2')

,1,2,3,4,5,6,7
model for acetate,,,,,,,
"gradient boosting, raw",0.548501,0.668077,3.022939e-01,0.124592,0.699858,0.593794,7.288118e-01
"random forest, raw",0.371436,0.556671,2.539804e-01,0.173076,0.669371,0.541002,7.453812e-01
"support vector, raw",0.035067,0.035100,1.204975e-01,0.086941,0.791448,0.968939,1.615791e-30
"neural net, raw",0.175968,0.026711,3.442920e-03,0.083997,0.819522,0.968588,1.325555e-01
"lasso, raw",0.100802,0.047517,1.280828e-32,0.000000,0.791448,0.248291,1.325555e-01
"gradient boosting, smooth",0.623236,0.614020,3.288885e-01,0.010633,0.918293,0.359991,7.651585e-01
"random forest, smooth",0.486779,0.612887,4.014225e-01,0.034168,0.525209,0.021086,8.226647e-01
"support vector, smooth",0.039870,0.021111,3.453743e-05,0.052304,0.614364,0.898623,0.000000e+00
"neural net, smooth",0.097399,0.052384,1.228485e-02,0.117982,0.882375,0.968939,1.325555e-01


,1,2,3,4,5,6,7
model for biomass,,,,,,,
"gradient boosting, raw",0.413946,0.648957,0.798046,0.362169,0.264585,0.430099,0.661652
"random forest, raw",0.454660,0.883734,0.911046,0.393696,0.000711,0.408818,0.551893
"support vector, raw",0.635186,0.850463,0.899274,0.378712,0.017580,0.310428,0.000000
"neural net, raw",0.221141,0.093387,0.286709,0.432206,0.019149,0.612325,0.594725
"lasso, raw",0.893117,0.834530,0.632329,0.145939,0.021304,0.609444,0.599886
"gradient boosting, smooth",0.357576,0.744789,0.858120,0.215717,0.254879,0.588165,0.668323
"random forest, smooth",0.464677,0.790367,0.807746,0.586141,0.583253,0.004077,0.593667
"support vector, smooth",0.679476,0.807224,0.907259,0.349971,0.002020,0.014889,0.000000
"neural net, smooth",0.099937,0.566189,0.393497,0.429382,0.021304,0.609444,0.641587


,1,2,3,4,5,6,7
model for butanol,,,,,,,
"gradient boosting, raw",0.774499,0.665385,0.896648,0.330809,0.891185,0.131727,0.885862
"random forest, raw",0.760712,0.743323,0.879584,0.433723,0.894864,0.847959,0.914101
"support vector, raw",0.729647,0.765166,0.796533,0.506721,0.944909,0.887491,0.969410
"neural net, raw",0.680524,0.589396,0.740020,0.518282,0.948886,0.894297,0.969410
"lasso, raw",0.894762,0.756912,0.754299,0.490832,0.944909,0.887491,0.969410
"gradient boosting, smooth",0.593498,0.866243,0.878145,0.483721,0.986779,0.789037,0.961841
"random forest, smooth",0.785812,0.820901,0.870092,0.477543,0.972472,0.901690,0.974594
"support vector, smooth",0.748420,0.760586,0.767589,0.513722,0.944909,0.887491,0.969410
"neural net, smooth",0.729937,0.798204,0.799572,0.536717,0.479226,0.778859,0.928878


,1,2,3,4,5,6,7
model for butyrate,,,,,,,
"gradient boosting, raw",0.459633,0.546265,0.036859,0.461035,0.806602,1.246242e-01,0.495154
"random forest, raw",0.564865,0.723706,0.343705,0.315740,0.910613,6.892939e-01,0.469415
"support vector, raw",0.110037,0.695151,0.713409,0.187758,0.750303,8.903575e-01,0.405259
"neural net, raw",0.185611,0.909758,0.752977,0.325527,0.751107,9.044856e-01,0.445707
"lasso, raw",0.144319,0.668764,0.735305,0.160623,0.750481,2.433781e-32,0.405259
"gradient boosting, smooth",0.408812,0.671880,0.428971,0.225033,0.988781,9.231306e-01,0.583161
"random forest, smooth",0.460798,0.837463,0.519680,0.304102,0.962432,9.719227e-01,0.635083
"support vector, smooth",0.073960,0.830114,0.629232,0.184941,0.750481,8.910693e-01,0.405259
"neural net, smooth",0.041432,0.884761,0.681037,0.238621,0.750481,8.944242e-01,0.441492


,1,2,3,4,5,6,7
model for ethanol,,,,,,,
"gradient boosting, raw",0.733319,0.813491,0.817550,0.275319,0.695843,0.395828,0.847556
"random forest, raw",0.705529,0.802124,0.907659,0.410464,0.453067,0.421844,0.891404
"support vector, raw",0.331342,0.843569,0.747833,0.883688,0.295355,0.418763,0.000000
"neural net, raw",0.104270,0.721172,0.697947,0.496402,0.307962,0.448161,0.922197
"lasso, raw",0.205940,0.858725,0.798209,0.890912,0.283825,0.418763,0.916285
"gradient boosting, smooth",0.548512,0.839963,0.236346,0.420368,0.810369,0.463972,0.783534
"random forest, smooth",0.651837,0.821122,0.365875,0.528331,0.543156,0.350648,0.911718
"support vector, smooth",0.161117,0.789449,0.713911,0.879550,0.298258,0.423839,0.000000
"neural net, smooth",0.075244,0.515663,0.406881,0.830175,0.363679,0.269823,0.916285


In [14]:
evaluate_models(smoothed_time_predictions, smooth_data, 'rmse')

,1,2,3,4,5,6,7
model for acetate,,,,,,,
"gradient boosting, raw",15.306899,14.518324,24.971868,20.977222,11.385379,12.505223,16.173283
"random forest, raw",17.815956,18.696770,26.191555,23.447740,8.183207,11.527848,19.047738
"support vector, raw",18.890460,26.518139,27.256239,22.688036,14.043738,17.282519,14.770709
"neural net, raw",21.197225,21.527156,29.762508,17.249838,13.559270,19.739908,215.729281
"lasso, raw",18.737520,27.508765,30.435476,23.603912,12.033316,15.270050,91.612675
"gradient boosting, smooth",16.712033,22.252645,22.583272,24.376324,6.854852,15.419678,15.111170
"random forest, smooth",17.191394,28.823283,20.235758,17.526591,9.026240,19.401712,16.765788
"support vector, smooth",19.076277,25.680734,29.428729,23.542168,13.202798,15.070245,17.091157
"neural net, smooth",24.084687,20.389233,28.186788,25.762045,11.145707,18.442036,183.618087


,1,2,3,4,5,6,7
model for biomass,,,,,,,
"gradient boosting, raw",0.074067,0.102054,0.062562,0.076795,0.044120,0.072037,0.050752
"random forest, raw",0.081558,0.120402,0.084765,0.068420,0.055206,0.088530,0.079608
"support vector, raw",0.092347,0.107977,0.024582,0.061634,0.045680,0.019255,1.058504
"neural net, raw",0.186650,0.178462,0.067572,0.170036,0.040599,0.055516,0.037391
"lasso, raw",0.080947,0.083146,0.107433,0.097797,0.051777,0.069536,0.042893
"gradient boosting, smooth",0.086814,0.114589,0.088670,0.076738,0.035852,0.062044,0.081199
"random forest, smooth",0.087588,0.097304,0.109746,0.062970,0.020227,0.050121,0.089950
"support vector, smooth",0.056589,0.061594,0.026619,0.059847,0.056470,0.041801,1.029819
"neural net, smooth",0.149540,0.140968,0.110747,0.145453,0.135752,0.114041,0.060967


,1,2,3,4,5,6,7
model for butanol,,,,,,,
"gradient boosting, raw",4.233223,3.421571,3.134694,4.582613,3.081225,2.794424,1.786223
"random forest, raw",4.102711,2.846080,3.552826,4.770351,3.892366,2.875300,1.821359
"support vector, raw",4.654410,2.953087,1.944642,4.556146,2.370439,2.012247,66.032092
"neural net, raw",5.151539,3.724122,3.338022,4.909691,1.003751,3.118562,3.382206
"lasso, raw",4.458595,2.332626,3.414702,4.132335,3.296540,3.345458,68.751630
"gradient boosting, smooth",6.063818,3.239771,3.358305,5.160494,3.223359,1.832249,0.807826
"random forest, smooth",4.175152,2.100306,4.901653,5.094548,4.352677,0.566030,0.803805
"support vector, smooth",5.258260,3.186987,1.954073,4.720511,2.003311,1.743534,65.082418
"neural net, smooth",5.111734,2.144070,2.057505,4.829320,1.219508,1.047662,4.192949


,1,2,3,4,5,6,7
model for butyrate,,,,,,,
"gradient boosting, raw",5.356248,2.693040,4.083755,7.898950,2.237840,6.343523,8.727143
"random forest, raw",3.808030,1.915376,2.867690,6.212155,2.317180,7.358070,8.257450
"support vector, raw",4.925363,4.559055,1.636569,4.636604,3.368802,3.973769,54.901189
"neural net, raw",4.275888,3.183572,2.974756,5.351575,4.606689,7.211962,15.124004
"lasso, raw",5.186964,5.458984,2.963627,6.025839,5.749715,6.689039,36.587317
"gradient boosting, smooth",5.259650,2.282372,1.868680,4.518311,1.921414,7.019167,8.475890
"random forest, smooth",5.259652,1.570404,2.745197,7.120359,1.167121,6.039961,8.275271
"support vector, smooth",4.984707,4.450255,2.008979,5.253737,3.784540,4.998248,88.915879
"neural net, smooth",6.242164,2.423580,2.796193,3.987424,3.265904,5.048346,14.382082


,1,2,3,4,5,6,7
model for ethanol,,,,,,,
"gradient boosting, raw",32.177033,13.369522,26.810449,36.953434,18.568066,36.241189,12.505441
"random forest, raw",34.309851,11.754146,13.166882,34.910174,21.403650,38.723131,5.305106
"support vector, raw",25.691333,15.054969,34.506002,35.192892,8.701936,13.900240,11.262807
"neural net, raw",23.837368,25.694572,26.414446,39.310413,10.150692,16.710522,276.645104
"lasso, raw",44.575728,7.203230,32.943622,30.392063,17.708714,22.555349,366.438362
"gradient boosting, smooth",36.146658,13.293878,29.260022,35.736656,17.271761,35.232326,12.398630
"random forest, smooth",31.525197,6.918298,24.969817,40.756713,19.610508,40.752570,9.681581
"support vector, smooth",40.029184,17.926281,35.028126,35.834706,6.317363,12.513565,11.309861
"neural net, smooth",24.771715,15.463711,31.524510,31.713571,3.677445,16.266712,230.689223


In [15]:
evaluate_models(smoothed_time_predictions, smooth_data, 'norm_rmse')

,1,2,3,4,5,6,7
model for acetate,,,,,,,
"gradient boosting, raw",0.410981,0.248765,0.423735,0.820896,0.325460,0.343065,0.302877
"random forest, raw",0.478348,0.320361,0.444432,0.917574,0.233923,0.316252,0.356707
"support vector, raw",0.507198,0.454377,0.462498,0.887845,0.401451,0.474124,0.276611
"neural net, raw",0.569133,0.368859,0.505025,0.675033,0.387602,0.541539,4.039961
"lasso, raw",0.503091,0.471351,0.516445,0.923686,0.343982,0.418914,1.715630
"gradient boosting, smooth",0.448708,0.381290,0.383204,0.953912,0.195951,0.423019,0.282987
"random forest, smooth",0.461579,0.493875,0.343371,0.685864,0.258022,0.532261,0.313973
"support vector, smooth",0.512187,0.440028,0.499362,0.921270,0.377412,0.413433,0.320066
"neural net, smooth",0.646659,0.349361,0.478288,1.008139,0.318609,0.505934,3.438615


,1,2,3,4,5,6,7
model for biomass,,,,,,,
"gradient boosting, raw",0.144707,0.199554,0.161570,0.203335,0.142710,0.247046,0.110409
"random forest, raw",0.159343,0.235431,0.218910,0.181160,0.178569,0.303606,0.173185
"support vector, raw",0.180423,0.211136,0.063484,0.163191,0.147754,0.066034,2.302755
"neural net, raw",0.364667,0.348960,0.174508,0.450215,0.131320,0.190388,0.081344
"lasso, raw",0.158150,0.162582,0.277451,0.258942,0.167475,0.238468,0.093314
"gradient boosting, smooth",0.169613,0.224064,0.228993,0.203184,0.115966,0.212774,0.176646
"random forest, smooth",0.171125,0.190266,0.283424,0.166730,0.065426,0.171884,0.195685
"support vector, smooth",0.110560,0.120440,0.068744,0.158460,0.182656,0.143352,2.240351
"neural net, smooth",0.292164,0.275644,0.286009,0.385125,0.439099,0.391095,0.132633


,1,2,3,4,5,6,7
model for butanol,,,,,,,
"gradient boosting, raw",0.588649,0.916302,0.888522,0.865926,2.326714,1.959793,0.358417
"random forest, raw",0.570501,0.762184,1.007041,0.901401,2.939227,2.016514,0.365467
"support vector, raw",0.647217,0.790841,0.551204,0.860925,1.789980,1.411235,13.249751
"neural net, raw",0.716345,0.997325,0.946155,0.927731,0.757959,2.187119,0.678661
"lasso, raw",0.619988,0.624681,0.967890,0.780842,2.489303,2.346246,13.795443
"gradient boosting, smooth",0.843202,0.867615,0.951904,0.975122,2.434043,1.284998,0.162095
"random forest, smooth",0.580574,0.562465,1.389363,0.962661,3.286820,0.396970,0.161289
"support vector, smooth",0.731185,0.853480,0.553878,0.891983,1.512752,1.222780,13.059193
"neural net, smooth",0.710810,0.574185,0.583195,0.912544,0.920883,0.734750,0.841341


,1,2,3,4,5,6,7
model for butyrate,,,,,,,
"gradient boosting, raw",0.849581,0.687384,1.638080,6.402522,0.270419,0.768743,0.637437
"random forest, raw",0.604010,0.488889,1.150291,5.035284,0.280007,0.891691,0.603130
"support vector, raw",0.781236,1.163674,0.656462,3.758216,0.407084,0.481563,4.010024
"neural net, raw",0.678220,0.812589,1.193237,4.337738,0.556669,0.873985,1.104669
"lasso, raw",0.822730,1.393376,1.188773,4.884265,0.694792,0.810614,2.672365
"gradient boosting, smooth",0.834259,0.582563,0.749567,3.662333,0.232183,0.850621,0.619085
"random forest, smooth",0.834259,0.400837,1.101156,5.771432,0.141034,0.731955,0.604432
"support vector, smooth",0.790649,1.135903,0.805844,4.258436,0.457321,0.605715,6.494483
"neural net, smooth",0.990100,0.618606,1.121612,3.232021,0.394650,0.611786,1.050478


,1,2,3,4,5,6,7
model for ethanol,,,,,,,
"gradient boosting, raw",1.896306,0.374079,0.582340,0.766374,3.870886,6.299547,0.779337
"random forest, raw",2.022000,0.328881,0.285993,0.723999,4.462020,6.730966,0.330613
"support vector, raw",1.514081,0.421238,0.749492,0.729862,1.814093,2.416180,0.701896
"neural net, raw",1.404820,0.718933,0.573739,0.815255,2.116115,2.904671,17.240477
"lasso, raw",2.627005,0.201546,0.715556,0.630298,3.691737,3.920635,22.836378
"gradient boosting, smooth",2.130250,0.371962,0.635546,0.741139,3.600645,6.124184,0.772681
"random forest, smooth",1.857891,0.193574,0.542360,0.845250,4.088203,7.083728,0.603355
"support vector, smooth",2.359061,0.501577,0.760833,0.743173,1.316981,2.175144,0.704829
"neural net, smooth",1.459884,0.432674,0.684732,0.657705,0.766637,2.827526,14.376514


In [16]:
def validate_models(pred_df_dict, raw_df, metric):
    species_set = ['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol']
    test_comp_set=[8, 9, 10]
    index_set = ['gradient boosting, raw', 'random forest, raw', 'support vector, raw', 'neural net, raw', 'lasso, raw', 'gradient boosting, smooth', 'random forest, smooth', 'support vector, smooth', 'neural net, smooth', 'lasso, smooth']
    
    for species in species_set:
        data = {}
        for test_comp in test_comp_set:
            data[test_comp] = []
            for model_name in pred_df_dict.keys():
                # print(model_name)
                if 'none' in model_name:
                    predicted_species_values = list(pred_df_dict[model_name].loc[test_comp][species])
                    measured_species_values = list(raw_df.loc[test_comp][species])

                    r2 = get_pearson_r2(measured_species_values, predicted_species_values)
                    rmse = get_rmse (measured_species_values, predicted_species_values)
                    norm_rmse = get_norm_rmse (measured_species_values, predicted_species_values)
                    
                    if metric == 'r2':
                        data[test_comp].append(r2)
                    elif metric == 'rmse':
                        data[test_comp].append(rmse)
                    elif metric == 'norm_rmse':
                        data[test_comp].append(norm_rmse)
                    else:
                        print('unknown metric')
        species_data = pd.DataFrame.from_dict(data)
        species_data[f'model for {species}'] = index_set
        species_data.set_index(f'model for {species}', inplace=True, drop=True)
        display(species_data)

In [17]:
validate_models(smoothed_time_predictions, smooth_data, 'r2')

,8,9,10
model for acetate,,,
"gradient boosting, raw",0.650986,0.773269,0.356549
"random forest, raw",0.470514,0.789059,0.294320
"support vector, raw",0.041596,0.005232,0.097053
"neural net, raw",0.022353,0.084618,0.359561
"lasso, raw",0.040719,0.031267,0.277238
"gradient boosting, smooth",0.638124,0.708814,0.322697
"random forest, smooth",0.300057,0.678735,0.245669
"support vector, smooth",0.033047,0.029893,0.136159
"neural net, smooth",0.021078,0.078325,0.331151


,8,9,10
model for biomass,,,
"gradient boosting, raw",0.316915,0.803194,0.804636
"random forest, raw",0.302009,0.756091,0.847974
"support vector, raw",0.234118,0.707693,0.014451
"neural net, raw",0.584348,0.025695,0.141561
"lasso, raw",0.705270,0.793511,0.659515
"gradient boosting, smooth",0.316057,0.769146,0.834122
"random forest, smooth",0.292504,0.694624,0.860583
"support vector, smooth",0.142264,0.737859,0.105595
"neural net, smooth",0.562381,0.433036,0.047149


,8,9,10
model for butanol,,,
"gradient boosting, raw",0.661756,0.743185,0.482463
"random forest, raw",0.837127,0.823782,0.475587
"support vector, raw",0.864086,0.875460,0.526504
"neural net, raw",0.693970,0.792849,0.707524
"lasso, raw",0.895087,0.876163,0.623665
"gradient boosting, smooth",0.755266,0.912930,0.472038
"random forest, smooth",0.673694,0.870335,0.410396
"support vector, smooth",0.877782,0.873395,0.580743
"neural net, smooth",0.551120,0.891251,0.485570


,8,9,10
model for butyrate,,,
"gradient boosting, raw",0.405974,0.045304,0.571431
"random forest, raw",0.656511,0.309174,0.675151
"support vector, raw",0.915058,0.527787,0.439651
"neural net, raw",0.914617,0.490592,0.564284
"lasso, raw",0.201284,0.556218,0.518009
"gradient boosting, smooth",0.746926,0.171872,0.784210
"random forest, smooth",0.753165,0.157680,0.665514
"support vector, smooth",0.127492,0.572454,0.571547
"neural net, smooth",0.159765,0.574928,0.559635


,8,9,10
model for ethanol,,,
"gradient boosting, raw",0.164022,0.655763,0.402006
"random forest, raw",0.275449,0.856890,0.390796
"support vector, raw",0.362514,0.948535,0.413129
"neural net, raw",0.081519,0.835501,0.479158
"lasso, raw",0.079644,0.918507,0.311875
"gradient boosting, smooth",0.243185,0.822900,0.511535
"random forest, smooth",0.156422,0.731851,0.411368
"support vector, smooth",0.084040,0.947677,0.386214
"neural net, smooth",0.078239,0.784071,0.151119


In [18]:
validate_models(smoothed_time_predictions, smooth_data, 'rmse')

,8,9,10
model for acetate,,,
"gradient boosting, raw",18.399515,11.279777,13.967399
"random forest, raw",18.062534,14.293517,12.149907
"support vector, raw",12.766360,14.043616,11.731608
"neural net, raw",14.461121,10.749568,15.869501
"lasso, raw",30.670039,27.472640,37.444873
"gradient boosting, smooth",18.931098,11.023591,10.376524
"random forest, smooth",24.904464,21.486031,12.145072
"support vector, smooth",15.597942,16.555919,10.357824
"neural net, smooth",26.561262,22.170079,20.693445


,8,9,10
model for biomass,,,
"gradient boosting, raw",0.968892,1.032450,0.794372
"random forest, raw",1.000432,1.057215,0.813322
"support vector, raw",1.929789,1.964812,1.891924
"neural net, raw",1.161363,1.213210,1.069334
"lasso, raw",0.308505,0.436941,0.077146
"gradient boosting, smooth",0.985631,1.039574,0.825164
"random forest, smooth",0.967512,1.015323,0.773584
"support vector, smooth",1.650273,1.692794,1.582637
"neural net, smooth",0.886451,0.958816,0.893978


,8,9,10
model for butanol,,,
"gradient boosting, raw",1.567843,2.091699,4.194532
"random forest, raw",1.881757,1.488035,6.573947
"support vector, raw",6.794090,8.700703,6.771344
"neural net, raw",1.458245,2.647729,4.311911
"lasso, raw",2.041780,3.454321,12.014504
"gradient boosting, smooth",1.193768,1.104932,4.658002
"random forest, smooth",2.148906,1.278591,4.462865
"support vector, smooth",5.927996,8.151481,4.583942
"neural net, smooth",3.649301,4.621034,4.685692


,8,9,10
model for butyrate,,,
"gradient boosting, raw",1.431975,1.898294,5.332683
"random forest, raw",1.788545,2.314250,4.662474
"support vector, raw",6.582488,6.867392,2.616390
"neural net, raw",3.264659,3.426207,7.109319
"lasso, raw",14.316072,16.901182,20.291825
"gradient boosting, smooth",3.433312,4.406064,3.296608
"random forest, smooth",2.292116,4.140844,4.083169
"support vector, smooth",11.758457,12.460218,3.723441
"neural net, smooth",13.692506,14.724420,13.605664


,8,9,10
model for ethanol,,,
"gradient boosting, raw",24.019890,21.221007,14.722621
"random forest, raw",27.312942,25.626179,35.671930
"support vector, raw",28.663671,26.428422,9.734209
"neural net, raw",36.271499,33.620580,7.029316
"lasso, raw",72.985515,81.778928,73.746454
"gradient boosting, smooth",23.978133,21.783701,13.377816
"random forest, smooth",27.846237,28.706470,10.957419
"support vector, smooth",29.216465,29.950347,8.624840
"neural net, smooth",45.062259,44.230737,23.464552


In [19]:
validate_models(smoothed_time_predictions, smooth_data, 'norm_rmse')

,8,9,10
model for acetate,,,
"gradient boosting, raw",0.555075,0.335714,0.275846
"random forest, raw",0.544909,0.425411,0.239952
"support vector, raw",0.385134,0.417973,0.231691
"neural net, raw",0.436262,0.319934,0.313411
"lasso, raw",0.925251,0.817654,0.739508
"gradient boosting, smooth",0.571111,0.328089,0.204929
"random forest, smooth",0.751315,0.639478,0.239856
"support vector, smooth",0.470557,0.492745,0.204559
"neural net, smooth",0.801298,0.659837,0.408680


,8,9,10
model for biomass,,,
"gradient boosting, raw",0.668394,0.688737,0.588674
"random forest, raw",0.690151,0.705257,0.602717
"support vector, raw",1.331272,1.310706,1.402021
"neural net, raw",0.801171,0.809320,0.792436
"lasso, raw",0.212823,0.291479,0.057169
"gradient boosting, smooth",0.679941,0.693489,0.611493
"random forest, smooth",0.667442,0.677311,0.573269
"support vector, smooth",1.138447,1.129246,1.172823
"neural net, smooth",0.611521,0.639617,0.662488


,8,9,10
model for butanol,,,
"gradient boosting, raw",0.556756,0.659161,0.908813
"random forest, raw",0.668230,0.468927,1.424351
"support vector, raw",2.412647,2.741868,1.467120
"neural net, raw",0.517837,0.834384,0.934245
"lasso, raw",0.725056,1.088566,2.603135
"gradient boosting, smooth",0.423919,0.348199,1.009231
"random forest, smooth",0.763097,0.402925,0.966952
"support vector, smooth",2.105089,2.568791,0.993185
"neural net, smooth",1.295902,1.456235,1.015231


,8,9,10
model for butyrate,,,
"gradient boosting, raw",0.609958,0.898586,0.585235
"random forest, raw",0.761842,1.095484,0.511683
"support vector, raw",2.803850,3.250781,0.287136
"neural net, raw",1.390601,1.621846,0.780212
"lasso, raw",6.098017,8.000424,2.226925
"gradient boosting, smooth",1.462440,2.085676,0.361786
"random forest, smooth",0.976340,1.960130,0.448107
"support vector, smooth",5.008585,5.898228,0.408629
"neural net, smooth",5.832405,6.970021,1.493153


,8,9,10
model for ethanol,,,
"gradient boosting, raw",0.645208,0.598409,1.046517
"random forest, raw",0.733664,0.722630,2.535641
"support vector, raw",0.769947,0.745253,0.691930
"neural net, raw",0.974304,0.948064,0.499660
"lasso, raw",1.960494,2.306076,5.242064
"gradient boosting, smooth",0.644086,0.614277,0.950925
"random forest, smooth",0.747989,0.809491,0.778878
"support vector, smooth",0.784795,0.844567,0.613073
"neural net, smooth",1.210436,1.247258,1.667913


In [20]:
smoothed_time_predictions['gradient boosting, smooth, test comp = none'].loc[[8,9,10]].to_csv(f'{cur_dir}/outputs/gb_smooth_val_conc.csv')

In [21]:
smoothed_time_predictions['gradient boosting, smooth, test comp = none'].loc[[7,8,9]]

acetate   biomass   butanol  butyrate    ethanol
composition trial time                                                    
7           1     0.0   27.036694  0.389823  1.538367  6.563626  19.843108
                  0.1   28.133542  0.389823  1.538367  7.226390  19.843108
                  0.2   30.913678  0.389823  1.538367  7.976210  19.843108
                  0.3   33.989511  0.389823  1.538367  8.765362  19.843108
                  0.4   36.250469  0.389823  1.538367  9.561328  19.843108
...                           ...       ...       ...       ...        ...
9           2     3.6   50.422132  0.570252  6.871510  8.275723  24.873748
                  3.7   49.768103  0.570252  7.460837  8.387392  24.873748
                  3.8   49.301465  0.570252  7.506028  8.713366  24.873748
                  3.9   48.288219  0.570252  7.882984  9.054769  24.873748
                  4.0   48.113887  0.570252  8.196995  9.108978  24.873748

[213 rows x 5 columns]

In [22]:
def get_feature_importances_conc(model):
    outputs = ['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol']
    features = ['time','acetate_0', 'biomass_0', 'butanol_0', 'butyrate_0', 'ethanol_0', 'CO', 'CO2', 'H2', 'flow rate']

    array_list = []

    for i in range(5):
        feature_importance_array = model.estimators_[i].steps[1][1].best_estimator_.feature_importances_
        array_list.append(list(feature_importance_array))
    df = pd.DataFrame(array_list, columns = features, index = outputs)  
    return df

In [23]:
model_list = [
    trained_models['gradient boosting, smooth, test comp = none'],
    trained_models['random forest, smooth, test comp = none'],
    ]

for model in model_list:
    display(get_feature_importances_conc(model))

,time,acetate_0,biomass_0,butanol_0,butyrate_0,ethanol_0,CO,CO2,H2,flow rate
acetate,0.471134,0.076699,0.041446,0.004612,0.010381,0.172301,0.144231,0.042428,0.016062,0.020707
biomass,0.299978,0.194535,0.468192,0.015522,0.009588,0.010639,0.000000,0.001546,0.000000,0.000000
butanol,0.752341,0.130130,0.010571,0.034643,0.046410,0.014013,0.006858,0.005033,0.000000,0.000000
butyrate,0.367743,0.026928,0.149309,0.009131,0.013255,0.380314,0.009782,0.016777,0.025304,0.001457
ethanol,0.400849,0.009113,0.053018,0.079843,0.029810,0.353430,0.000626,0.003855,0.064043,0.005413


,time,acetate_0,biomass_0,butanol_0,butyrate_0,ethanol_0,CO,CO2,H2,flow rate
acetate,0.493171,0.009259,0.069724,0.042933,0.032602,0.073414,0.267188,0.004167,0.002152,0.005390
biomass,0.332543,0.110484,0.508792,0.009829,0.006319,0.011543,0.007388,0.006376,0.005624,0.001101
butanol,0.802446,0.197554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
butyrate,0.382935,0.018804,0.140303,0.038745,0.006926,0.321230,0.023670,0.012649,0.045501,0.009238
ethanol,0.416112,0.011902,0.046451,0.048042,0.031755,0.306002,0.001104,0.003401,0.110356,0.024874


In [24]:
print('gradient boosting, smooth, test comp = none')
display(get_feature_importances_conc(trained_models['gradient boosting, smooth, test comp = none']))
print('random forest, smooth, test comp = none')
display(get_feature_importances_conc(trained_models['random forest, smooth, test comp = none']))

gradient boosting, smooth, test comp = none


,time,acetate_0,biomass_0,butanol_0,butyrate_0,ethanol_0,CO,CO2,H2,flow rate
acetate,0.471134,0.076699,0.041446,0.004612,0.010381,0.172301,0.144231,0.042428,0.016062,0.020707
biomass,0.299978,0.194535,0.468192,0.015522,0.009588,0.010639,0.000000,0.001546,0.000000,0.000000
butanol,0.752341,0.130130,0.010571,0.034643,0.046410,0.014013,0.006858,0.005033,0.000000,0.000000
butyrate,0.367743,0.026928,0.149309,0.009131,0.013255,0.380314,0.009782,0.016777,0.025304,0.001457
ethanol,0.400849,0.009113,0.053018,0.079843,0.029810,0.353430,0.000626,0.003855,0.064043,0.005413


random forest, smooth, test comp = none


,time,acetate_0,biomass_0,butanol_0,butyrate_0,ethanol_0,CO,CO2,H2,flow rate
acetate,0.493171,0.009259,0.069724,0.042933,0.032602,0.073414,0.267188,0.004167,0.002152,0.005390
biomass,0.332543,0.110484,0.508792,0.009829,0.006319,0.011543,0.007388,0.006376,0.005624,0.001101
butanol,0.802446,0.197554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
butyrate,0.382935,0.018804,0.140303,0.038745,0.006926,0.321230,0.023670,0.012649,0.045501,0.009238
ethanol,0.416112,0.011902,0.046451,0.048042,0.031755,0.306002,0.001104,0.003401,0.110356,0.024874
